In [1]:
import numpy as np 
import cv2
from ultralytics import YOLO
import supervision as sv


In [44]:
detector = YOLO("yolov8s.pt")
classifier = YOLO("models/pose_classifier.pt")
byte_tracker = sv.ByteTrack()

In [4]:
classes = ["fall", "standing", "sitting"]

In [15]:
MAX_FALL=30
MAX_NOT_MOVING = 10

In [6]:
res = classifier("data/stand.jpg")


image 1/1 c:\Projects\PeopleFallDetection\data\stand.jpg: 224x224 no fall 0.67, sitting 0.30, fall 0.03, 20.9ms
Speed: 1.7ms preprocess, 20.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


In [8]:
res[0].probs.data

tensor([0.0294, 0.6745, 0.2962])

In [41]:
def compute_offsets(bbox1, bbox2):
    '''
    compute differences between previous and current bboxes
    '''
    dx = (bbox2[0]-bbox1[0])**2 + (bbox2[2]-bbox1[2])**2
    dy = (bbox2[1]-bbox1[1])**2 + (bbox2[3]-bbox1[3])**2
    
    return [dx, dy]

In [8]:
if compute_offsets([2,3, 5,6], [2,3, 5,6])== [0, 0]:
    print("true")

true


In [46]:
cap = cv2.VideoCapture("data/people_falling_3.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)


cap_writer = cv2.VideoWriter('data/result_detection.avi', cv2.VideoWriter_fourcc(*'MJPG'),
                        25, (width, height))

success, frame = cap.read()
frame_num=0
total_fps=0

frame_num=0

OBJECTS={}

while success:
    
    
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
  
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
    alarm = False
    
       
    if frame_num%5==0:
        #renew predictions every 5 frame
        detections = detector(frame)[0]
        detections = sv.Detections.from_ultralytics(detections)  
                
    detections = byte_tracker.update_with_detections(detections)
        
       
    bboxes = detections.xyxy
    confidences = detections.confidence
    class_ids = detections.class_id
    tracker_ids = detections.tracker_id
    
            
    for pred_box, label, tracker_id in zip(bboxes, class_ids, tracker_ids):
        
        if label == 0:
            bbox = [int(pred_box[0]), int(pred_box[1]), int(pred_box[2]), int(pred_box[3])]
            width = bbox[2]-bbox[0]
            height = bbox[3]-bbox[1]
            ROI = frame[bbox[1]:bbox[1]+height, 
                    bbox[0]:bbox[0]+width]
        
            results = classifier(ROI)
            second_class = classes[np.argmax(results[0].probs.data)]

            if tracker_id not in OBJECTS:
                OBJECTS[tracker_id] = {
                    "class_id": label,
                    "last_bbox": np.array(bbox),
                    "bbox": np.array(bbox),
                    "second_class": second_class,
                    "fall": 0,
                    "bbox_offsets": [0, 0],
                    "not_moving": 0
                    
                }
            else:
                object = OBJECTS[tracker_id] 
                object['class_id']=label
                object['last_bbox'] = OBJECTS[tracker_id]['bbox']
                object['bbox']=np.array(bbox)
                object['second_class']=second_class
                object['bbox_offsets'] = compute_offsets(object['last_bbox'], object['bbox'])
                if object['second_class']=="fall": #if fall
                    object['fall']+=1
                    if object['bbox_offsets']==[0, 0]:
                        object['not_moving']+=1
                    else:
                        object['not_moving']=0    
                else:
                    object['fall']=0    
                    object['not_moving']=0
        
            
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color=(0,0,255), thickness=2)
            cv2.putText(frame, "id: "+str(tracker_id), (bbox[0], bbox[1]-10), 
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
        
        #check falling objects
        for ind, object in OBJECTS.items():
            if object['fall']>=MAX_FALL:
                 cv2.putText(frame, "ALARM!", (20, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=5) 
                 if object['not_moving']>=MAX_NOT_MOVING:
                     cv2.putText(frame, "Not moving!", (150, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3)
        
             
                
                    
    #resize frame for videowriter
    #frame = cv2.resize(frame, dsize=(640, 640), interpolation=cv2.INTER_CUBIC) 
    cap_writer.write(frame)      
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    success, frame = cap.read()
    frame_num+=1
    




0: 384x640 3 persons, 334.1ms
Speed: 0.0ms preprocess, 334.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 224x224 fall 0.63, no fall 0.32, sitting 0.05, 9.3ms
Speed: 0.0ms preprocess, 9.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 0.66, no fall 0.29, sitting 0.05, 15.3ms
Speed: 1.0ms preprocess, 15.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 0.69, no fall 0.27, sitting 0.04, 17.7ms
Speed: 0.0ms preprocess, 17.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 0.62, no fall 0.33, sitting 0.04, 3.7ms
Speed: 1.2ms preprocess, 3.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 0.73, no fall 0.24, sitting 0.03, 13.4ms
Speed: 1.0ms preprocess, 13.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 384x640 3 persons, 357.0ms
Speed: 0.0ms preprocess, 357.0ms inference, 0.0ms postprocess per ima

In [47]:
OBJECTS

{10025: {'class_id': 0,
  'last_bbox': array([437, 261, 540, 451]),
  'bbox': array([438, 261, 540, 451]),
  'second_class': 'standing',
  'fall': 0,
  'bbox_offsets': [1, 0],
  'not_moving': 0}}

In [26]:
for object in OBJECTS.items():
    print(object)

(231, {'class_id': 0, 'bbox': array([203, 101, 384, 435]), 'second_class': 'standing', 'fall': 0})


In [2]:
d={"1":1, "2":2}

In [3]:
d.values()

dict_values([1, 2])

### Training YOLO cls

In [8]:
DATASET_FOLDER = "datasets"

In [6]:
model = YOLO("yolov8s-cls.pt")

100%|██████████| 12.2M/12.2M [00:00<00:00, 14.7MB/s]


In [9]:
model.train(data=DATASET_FOLDER, epochs=50, imgsz=224)

New https://pypi.org/project/ultralytics/8.0.211 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.209  Python-3.8.9 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
engine\trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=datasets, epochs=50, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False

KeyboardInterrupt: 

In [3]:
!yolo classify train imgsz=224 epochs=50 data={DATASET_FOLDER} model=yolov8s-cls.pt --project yolov8s-people_fall_3_classes device=0

WARNING  '--project' does not require leading dashes '--', updating to 'project'.
Traceback (most recent call last):
  File "C:\Users\Alena_Prakonina\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Alena_Prakonina\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Projects\PeopleFallDetection\.venv\Scripts\yolo.exe\__main__.py", line 7, in <module>
  File "c:\projects\peoplefalldetection\.venv\lib\site-packages\ultralytics\cfg\__init__.py", line 378, in entrypoint
    raise SyntaxError(f"'{colorstr('red', 'bold', a)}' is a valid YOLO argument but is missing an '=' sign "
SyntaxError: 'project' is a valid YOLO argument but is missing an '=' sign to set its value, i.e. try 'project=None'

    Arguments received: ['yolo', 'classify', 'train', 'imgsz=224', 'epochs=50', 'data=datasets/train/', 'model=yolov8s-cls.pt', '--projec

In [2]:
def check_not_moving(frame, not_moving):
    if not_moving>=10:
        cv2.putText(frame, "Not moving",
                    (150, 50),
                    cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3) 

In [4]:
cap = cv2.VideoCapture("data/me.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)
    
backSub = cv2.createBackgroundSubtractorMOG2()

not_moving=0

while(cap.isOpened()):
    
    success, frame_prev = cap.read()
      
    
    fgMask_prev = backSub.apply(frame_prev)
    
    frame_prev = cv2.resize(frame_prev, dsize=(width, height))
    fgMask_prev = cv2.resize(fgMask_prev, dsize=(width, height))
    
    check_not_moving(frame_prev, not_moving) 
    
    cv2.imshow('Frame', frame_prev)
    cv2.imshow('FG Mask', fgMask_prev)
    
    
    success, frame_next = cap.read()
    
    fgMask_next = backSub.apply(frame_next)
    
    frame_next = cv2.resize(frame_next, dsize=(width, height))
    fgMask_next = cv2.resize(fgMask_next, dsize=(width, height))
    
    diff = cv2.absdiff(fgMask_prev, fgMask_next)   
    thresh_diff = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)[1]
    
    
    cv2.imshow("difference", diff)
    
    # Calculate the difference between the 2 images
    total_pixels = frame_prev.shape[0] * frame_prev.shape[1] * 1.0
    diff_on_pixels = cv2.countNonZero(thresh_diff) * 1.0
    difference_measure = diff_on_pixels / total_pixels
    
    print(difference_measure)
    
    if difference_measure<=0.03:
        not_moving+=1   
    else:
        not_moving = 0    
          
    check_not_moving(frame_next, not_moving) 
    
       
    cv2.imshow('Frame', frame_next)
    cv2.imshow('FG Mask', fgMask_next)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    
    success, frame = cap.read()
    

0.9987037037037036
0.03327739197530864
0.0238695987654321
0.01863425925925926
0.03042631172839506
0.023227237654320988
0.01912229938271605
0.020482253086419752
0.028512731481481483
0.02947530864197531
0.030495756172839505
0.03898533950617284
0.026786265432098767
0.03362654320987654
0.027662037037037037
0.03857638888888889
0.0510570987654321
0.05089699074074074
0.06965663580246914
0.08076774691358024
0.0691724537037037
0.07252893518518519
0.06911651234567902
0.07480902777777777
0.06703317901234568
0.06967785493827161
0.06793788580246914
0.0655420524691358
0.06500964506172839
0.06569251543209877
0.0619579475308642
0.0575462962962963
0.06144868827160494
0.06543016975308642
0.0654320987654321
0.060271990740740744
0.06544753086419754
0.06255979938271605
0.06515432098765432
0.06457947530864197
0.057629243827160496
0.05886766975308642
0.06327160493827161
0.06466242283950617
0.0619849537037037
0.06371527777777777
0.06276427469135802
0.06542631172839507
0.06360725308641975
0.0676466049382716
0.

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 